In [1]:
import numpy as np
import pandas as pd

from urllib.parse import urljoin
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup, SoupStrainer
from os.path import basename, dirname
from decimal import Decimal

pd.set_option('display.max_rows', 500)

In [503]:
def clean_soup(soup):

    table=soup.new_tag('table')
    thead=soup.new_tag('thead')
    tbody=soup.new_tag('tbody')
                   
    for x in soup.find_all("th"):
        if "colspan" in x.attrs:
            continue
            
        x.attrs=[]

        th_string=""
        for y in x:
            if y.string != None:
                th_string += ' '+y.string.replace('\n','')

        th_string = ' '.join(th_string.split())

        th=soup.new_tag('th')
        th.string=th_string  
        thead.append(th)
        
    for x in soup.find_all("tr"):
        if x.find('th'): x.decompose()

    for x in soup.find_all("th"):
        x.decompose()

    html_orig=soup.find('html')
    body_orig=soup.find('body')
    head_orig=soup.find('head')
    table_orig=soup.find('table')

    th_all=soup.find_all('th')
    tr_all=soup.find_all('tr')
    th_last=None
    tr_last=None

    if th_all!=None:
        th_last=th_all[-1]
    if tr_all!=None:
        tr_last=tr_all[-1]

    if table_orig!=None:
        if th_last not in table_orig('th') or tr_last not in table_orig('tr'):
            print('Remove table')
            table_orig.replaceWithChildren()

    if body_orig!=None:
        if th_last not in body_orig('th') or tr_last not in body_orig('tr'):
            print('Remove body')
            body_orig.replaceWithChildren()

    if html_orig!=None:
        if th_last not in html_orig('th') or tr_last not in html_orig('tr'):
            print('Remove html')
            html_orig.replaceWithChildren()        
        
        
        
    if thead_orig!=None: thead_orig.decompose()

        
    for th in soup.find_all('th'):
        thead.append(th)
    table.append(thead)

    for tr in soup.find_all('tr'):
        if tr.find('th'):
            tr.decompose()

    for tr in soup.find_all('tr'):
        tbody.append(tr)
    table.append(tbody)

    soup=BeautifulSoup('<html><body>', 'html.parser')
    soup.body.insert(0,table)
    soup
        
        
        
        
    if len(thead)>0:
        soup.table.insert(0,thead)
        
    return(soup)

def scrape_url(url): #, url_list=[], url_root=''):

    result_data=[]
    result_url=[]
    url_list=[]
    url_root=dirname(url)
    it=[0]
    
    def scrape_url(url, url_list, url_root, it):
        
        url_base=dirname(url)
        url_list.append(url)
        
        if not url_root in url_base:
            return
    
        try:
            content=urlopen(url).read()
            soup=BeautifulSoup(content, 'html.parser')
            soup=clean_soup(soup)
        except HTTPError as e:
            #print(e.code)
            #print(e.msg)
            return
            
        try:
            data=pd.read_html(str(soup))
            if (it[-1] % 10 == 0):
                print('Iteration: '+str(it[-1])+' - Tables: '+str(len(data))+' - URL: '+ url)
            result_url.append(url)
            result_data.append(data[0])
            it.append(it[-1]+1)
        except:
            print('No tables were found - URL: '+ url)
        
    
        for link in soup.find_all('a'):
            url=link.get('href')
            if not 'http' in url:
                url = urljoin(url_base+'/index.html', url, False)
        
            if not (url in url_list):
                scrape_url(url, url_list, url_base, it)
                 
    scrape_url(url, url_list, url_root, it)

    print("Number of processed tables: "+str(len(result_data)))
    
    return result_url, result_data

In [299]:
conicyt_url="http://www.conicyt.cl/transparencia/transferencias/transferencias_historico.html"
#conicyt_url="http://www.conicyt.cl/transparencia/transferencias/2008/transferencias.html"
#conicyt_url="http://www.conicyt.cl/transparencia/transferencias/2010/transferencias_meses_2010.html"
#conicyt_url="http://www.conicyt.cl/transparencia/transferencias/2015/01-enero/transferencias.html"

data_url, data = scrape_url(conicyt_url)

No tables were found - URL: http://www.conicyt.cl/transparencia/transferencias/transferencias_historico.html
No tables were found - URL: http://www.conicyt.cl/transparencia/transferencias/2015/transferencias_meses_2015.html
Iteration: 0 - Tables: 1 - URL: http://www.conicyt.cl/transparencia/transferencias/2015/01-enero/transferencias.html
Iteration: 10 - Tables: 1 - URL: http://www.conicyt.cl/transparencia/transferencias/2015/04-abril/transferencias.html
Iteration: 20 - Tables: 1 - URL: http://www.conicyt.cl/transparencia/transferencias/2015/05-mayo/transferencias-4.html
Iteration: 30 - Tables: 1 - URL: http://www.conicyt.cl/transparencia/transferencias/2015/06-junio/transferencias-5.html
Iteration: 40 - Tables: 1 - URL: http://www.conicyt.cl/transparencia/transferencias/2015/07-julio/transferencias-5.html
Iteration: 50 - Tables: 1 - URL: http://www.conicyt.cl/transparencia/transferencias/2015/07-julio/transferencias-15.html
Iteration: 60 - Tables: 1 - URL: http://www.conicyt.cl/transp

In [506]:
conicyt_url="http://www.conicyt.cl/transparencia/transferencias/2006/transferencias.html"
data_url, data = scrape_url(conicyt_url)

Remove table
Remove body
Remove html
3


AttributeError: 'NoneType' object has no attribute 'insert'

In [505]:
data[0].head()

,Nº correlativo,Fecha de la transferencia (dd/mm/aaaa),Utilidad o finalidad,Respaldo jurídico,Monto,RUT,Razón social,Región
0,1,22-01-2007,"Proyecto prog. EXPLORA 10° Concurso, ED10/002",Decreto 231 Ministerio de Educación año 2001,1.719.000,81698900-0,Pontificia Univ. Católica de Chile (PUC),XIII
1,2,22-01-2007,"Proyecto prog. EXPLORA 10° Concurso, ED10/041",Decreto 231 Ministerio de Educación año 2001,5.330.279,70791800-4,Universidad de Antofagasta,II
2,3,23-01-2007,"Proyecto prog. EXPLORA 9° Concurso, ED09/04/006",Decreto 231 Ministerio de Educación año 2001,7.098.296,61312000-9,Instituto de Investigaciones Agropecuarias (INIA),XIII
3,4,24-01-2007,"Proyecto prog. FONDECYT FONDAP, 15000001",Decreto Fuerza de Ley N° 33 Ministerio de Educ...,150.000.000,60910000-1,Universidad de Chile,XIII
4,5,24-01-2007,"Proyecto prog. FONDECYT FONDAP, 15010001",Decreto Fuerza de Ley N° 33 Ministerio de Educ...,300.000.000,81698900-0,Pontificia Univ. Católica de Chile (PUC),XIII


### Chequeamos si el nombre de las columnas es consistente para todos los Dataframe

In [300]:
data[0].head()

,Mes de la transferencia,Fecha de la transferencia (dd/mm/aaaa),Utilidad o finalidad de la transferencia,Respaldo jurídico de la transferencia,Imputación presupuestaria,Unidad monetaria,Monto,"RUT, si receptor es persona jurídica","Razón social de la persona jurídica, si corresponde","Apellido paterno, si receptor es persona natural","Apellido materno, si receptor es persona natural","Nombres, si receptor es persona natural",Región de la organización que recibe el aporte
0,ENERO,29-01-2015,Honorarios,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,2403163,PESOS,44564,NO APLICA,NO APLICA,Calisaya,Benavides,Ximena del Carmen,RM
1,ENERO,29-01-2015,Honorarios,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,2403163,PESOS,1049400,NO APLICA,NO APLICA,ABARZUA,RAMIREZ,RODRIGO,RM
2,ENERO,29-01-2015,Honorarios,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,2403163,PESOS,1495872,NO APLICA,NO APLICA,CALISAYA,BENAVIDES,XIMENA DEL CARMEN,RM
3,ENERO,29-01-2015,Honorarios,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,2403163,PESOS,1177778,NO APLICA,NO APLICA,CIFUENTES,QUEZADA,JOSE LUIS,RM
4,ENERO,29-01-2015,Honorarios,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,2403163,PESOS,1424640,NO APLICA,NO APLICA,DONAT,RODRIGUEZ,EVELYN DANIELA,RM


In [301]:
data[2084].head()

,Nº correlativo,Fecha de la transferencia (dd/mm/aaaa),Utilidad o finalidad,Respaldo jurídico,Monto,RUT,Razón social,Región
0,1.0,15-01-2008,"Proyecto progr.BECAS, ARANCELES, FACTURA N° 50...",Decreto Supremo N° 29 del 29-12-1988,950.000,60910000-1,UNIVERSIDAD DE CHILE,XIII
1,2.0,15-01-2008,"Proyecto progr.BECAS, ARANCELES, FACTURA N° 50...",Decreto Supremo N° 29 del 29-12-1988,300.000,60910000-1,UNIVERSIDAD DE CHILE,XIII
2,3.0,17-01-2008,"Proyecto progr.BECAS, ARANCELES, FACT N° 50592...",Decreto Supremo N° 29 del 29-12-1988,50.825.000,60910000-1,UNIVERSIDAD DE CHILE,XIII
3,4.0,18-01-2008,"Proyecto progr.DRI, TALLER DE CIENCIAS SOCIALE...",Decreto Supremo N°172 del 2001 del Ministerio ...,390.162,65483190-4,FUNDACION CENTRO CULTURAL PALACIO DE LA MONEDA,XIII
4,5.0,22-01-2008,"Proyecto progr.BECAS, ARANCELES, FACTURA N° 10...",Decreto Supremo N° 29 del 29-12-1988,150.000,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,XIII


In [303]:
conicyt=pd.concat(data)
print(conicyt.columns)
conicyt.head()

Index(['Apellido materno, si el receptor es persona natural',
       'Apellido materno, si receptor es persona natural',
       'Apellido paterno, si el receptor es persona natural',
       'Apellido paterno, si receptor es persona natural',
       'Fecha de la transferencia (dd/mm/aaaa)', 'Imputación presupuestaria',
       'Mes de la transferencia', 'Monto',
       'Nombres si el receptor es persona natural',
       'Nombres, si receptor es persona natural', 'Nº correlativo', 'RUT',
       'RUT (si es persona jurídica)', 'RUT, si receptor es persona jurídica',
       'Razón social', 'Razón social de la persona jurídica',
       'Razón social de la persona jurídica, si corresponde', 'Región',
       'Región de la organización que recibe el aporte', 'Respaldo jurídico',
       'Respaldo jurídico de la transferencia', 'Unidad monetaria',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Utilidad o finalidad',
       'Utilidad o finalidad de la transferencia'],
      dtype='object')


,"Apellido materno, si el receptor es persona natural","Apellido materno, si receptor es persona natural","Apellido paterno, si el receptor es persona natural","Apellido paterno, si receptor es persona natural",Fecha de la transferencia (dd/mm/aaaa),Imputación presupuestaria,Mes de la transferencia,Monto,Nombres si el receptor es persona natural,"Nombres, si receptor es persona natural",...,Región,Región de la organización que recibe el aporte,Respaldo jurídico,Respaldo jurídico de la transferencia,Unidad monetaria,Unnamed: 3,Unnamed: 4,Unnamed: 5,Utilidad o finalidad,Utilidad o finalidad de la transferencia
0,NaN,Benavides,NaN,Calisaya,29-01-2015,2403163,ENERO,44564,NaN,Ximena del Carmen,...,NaN,RM,NaN,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,PESOS,NaN,NaN,NaN,NaN,Honorarios
1,NaN,RAMIREZ,NaN,ABARZUA,29-01-2015,2403163,ENERO,1049400,NaN,RODRIGO,...,NaN,RM,NaN,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,PESOS,NaN,NaN,NaN,NaN,Honorarios
2,NaN,BENAVIDES,NaN,CALISAYA,29-01-2015,2403163,ENERO,1495872,NaN,XIMENA DEL CARMEN,...,NaN,RM,NaN,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,PESOS,NaN,NaN,NaN,NaN,Honorarios
3,NaN,QUEZADA,NaN,CIFUENTES,29-01-2015,2403163,ENERO,1177778,NaN,JOSE LUIS,...,NaN,RM,NaN,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,PESOS,NaN,NaN,NaN,NaN,Honorarios
4,NaN,RODRIGUEZ,NaN,DONAT,29-01-2015,2403163,ENERO,1424640,NaN,EVELYN DANIELA,...,NaN,RM,NaN,DECRETO 231 MINISTERIO DE EDUCACION AÑO 2001,PESOS,NaN,NaN,NaN,NaN,Honorarios


In [318]:
for i, df in enumerate(data):
    if 'Unnamed: 3' in df.columns.values:
        print('Table id=', i)

Table id= 2086


In [324]:
print(data_url[2086])
data[2086].head()

http://www.conicyt.cl/transparencia/transferencias/2006/transferencias.html


,RUT,Razón social,Región,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Nº correlativo,Fecha de la transferencia (dd/mm/aaaa),Utilidad o finalidad,Respaldo jurídico,Monto,Receptor


In [493]:
url='http://www.conicyt.cl/transparencia/transferencias/2006/transferencias.html'
content=urlopen(url).read()
soup=BeautifulSoup(content, 'html.parser')
#soup=clean_soup(soup)
#soup

In [494]:
html_orig=soup.find('html')
body_orig=soup.find('body')
head_orig=soup.find('head')
table_orig=soup.find('table')

th_all=soup.find_all('th')
tr_all=soup.find_all('tr')
th_last=None
tr_last=None

if th_all!=None:
    th_last=th_all[-1]
if tr_all!=None:
    tr_last=tr_all[-1]

if table_orig!=None:
    if th_last not in table_orig('th') or tr_last not in table_orig('tr'):
        print('Remove table')
        table_orig.replaceWithChildren()

if body_orig!=None:
    if th_last not in body_orig('th') or tr_last not in body_orig('tr'):
        print('Remove body')
        body_orig.replaceWithChildren()

if html_orig!=None:
    if th_last not in html_orig('th') or tr_last not in html_orig('tr'):
        print('Remove html')
        html_orig.replaceWithChildren()

table=soup.new_tag('table')
thead=soup.new_tag('thead')
tbody=soup.new_tag('tbody')

for th in soup.find_all('th'):
    thead.append(th)
table.append(thead)

for tr in soup.find_all('tr'):
    if tr.find('th'):
        tr.decompose()

for tr in soup.find_all('tr'):
    tbody.append(tr)
table.append(tbody)

soup=BeautifulSoup('<html><body>', 'html.parser')
soup.body.insert(0,table)
soup

Remove table
Remove body
Remove html


<html><body><table><thead><th rowspan="2" width="7%">Nº correlativo</th><th nowrap="nowrap" rowspan="2" width="9%">Fecha de la<br/>
        transferencia<br/> <b>(dd/mm/aaaa)</b> </th><th rowspan="2" width="15%">Utilidad o finalidad </th><th rowspan="2" title="Arriendo, compra , lisssing, etc " width="22%">Respaldo 
        jurídico </th><th rowspan="2" title="" width="11%">Monto</th><th colspan="3">Receptor</th><th nowrap="nowrap" width="10%">RUT</th><th width="16%">Razón social </th><th width="10%">Región</th></thead><tbody><tr>



</tr><tr>
<td>1</td>
<td>27-03-2006</td>
<td>Proyecto Progr. BID, BIO Acuicultura, AQ02I1003</td>
<td>Decreto Supremo N°172 del 2001 del Ministerio de Economía autoriza operación 
      de préstamo entre Chile y el BID y establece como Institución Ejecutora 
      al MINECON a través de su Subsecretaría / Contrato de Préstamo N°1286/0C-CH 
      entre el Ministerio de Economía y el BID / Decreto Supremo N°223 del 30 
      de abril de 2001 del Ministerio d

In [496]:
tr_all= soup.find_all('tr')
tr_all[0]

<tr>



</tr>

In [486]:
soup=clean_soup(soup)
df=pd.read_html(str(soup))
df[0].head()

thead:  <thead><th>RUT</th><th>Razón social</th><th>Región</th><th>Nº correlativo</th><th>Fecha de la transferencia (dd/mm/aaaa)</th><th>Utilidad o finalidad</th><th>Respaldo jurídico</th><th>Monto</th></thead>


,RUT,Razón social,Región,Nº correlativo,Fecha de la transferencia (dd/mm/aaaa),Utilidad o finalidad,Respaldo jurídico,Monto
0,1,27-03-2006,"Proyecto Progr. BID, BIO Acuicultura, AQ02I1003",Decreto Supremo N°172 del 2001 del Ministerio ...,21.161.737,60910000-1,Universidad de Chile,XIII
1,2,27-03-2006,"Proyecto Progr. BID, BIO Genoma, G02S1001",Decreto Supremo N°172 del 2001 del Ministerio ...,5.757.430,81698900-0,Pontificia Univ. Católica de Chile (PUC),XIII
2,3,27-03-2006,"Proyecto Progr. BID, BIO Genoma, G02S1001",Decreto Supremo N°172 del 2001 del Ministerio ...,9.300.965,73124100-7,Fundación Ciencia para la Vida,XIII
3,4,27-03-2006,"Proyecto Progr. BID, BIO Genoma, G02S1001",Decreto Supremo N°172 del 2001 del Ministerio ...,3.042.473,60910000-1,Universidad de Chile,XIII
4,5,27-03-2006,"Proyecto Progr. BID, BIO Acuicultura, AQ02I1031",Decreto Supremo N°172 del 2001 del Ministerio ...,29.400.677,70772100-6,Universidad de Los Lagos,X


In [481]:
df.head()

AttributeError: 'list' object has no attribute 'head'

In [364]:
table_orig

<table border="0" cellpadding="0" cellspacing="0" class="tabla" width="91%"><thead><th>RUT</th><th>Razón social</th><th>Región</th><th>Nº correlativo</th><th>Fecha de la transferencia (dd/mm/aaaa)</th><th>Utilidad o finalidad</th><th>Respaldo jurídico</th><th>Monto</th></thead>






</table>

In [339]:
url="http://www.conicyt.cl/transparencia/transferencias/2015/01-enero/transferencias.html"
content=urlopen(url).read()
soup=BeautifulSoup(content, 'html.parser')
soup=clean_soup(soup)
soup

thead:  <thead><th>Mes de la transferencia</th><th>Fecha de la transferencia (dd/mm/aaaa)</th><th>Utilidad o finalidad de la transferencia</th><th>Respaldo jurídico de la transferencia</th><th>Imputación presupuestaria</th><th>Unidad monetaria</th><th>Monto</th><th>RUT, si receptor es persona jurídica</th><th>Razón social de la persona jurídica, si corresponde</th><th>Apellido paterno, si receptor es persona natural</th><th>Apellido materno, si receptor es persona natural</th><th>Nombres, si receptor es persona natural</th><th>Región de la organización que recibe el aporte</th></thead>


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
   "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="es" xml:lang="es" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o?o:e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({QJf3ax:[function(t,e){function n(t){function e(e,n,a){t&&t(e,n,a),a||(a={});for(var c=u(e),f=c.length,s=i(a,o,r),p=0;f>p;p++)c[p].apply(s,n);return s}function a(t,e){f[t]=u(t).concat(e)}function u(t){return f[t]||[]}function c(){return n(e)}var f={};return{on:a,emit:e,create:c,listeners:u,_events:f}}function r(){return{}}var o="nr@context",i=t("gos");e.exports=n()},{gos:"7eSDFh"}],ee:[function

In [313]:
np.isnan(conicyt['Unnamed: 3'].values[0])

True

In [59]:
for i in range(len(data)):
    data[i].rename(columns={'Apellido paterno, si el receptor es persona natural':'Apellido paterno',
                            'Apellido paterno, si receptor es persona natural':'Apellido paterno',
                            'Apellido materno, si el receptor es persona natural':'Apellido materno',
                            'Apellido materno, si receptor es persona natural':'Apellido materno',
                            'Nombres si el receptor es persona natural':'Nombres',
                            'Nombres, si receptor es persona natural':'Nombres',
                            'Fecha de la transferencia(dd/mm/aaaa)':'Fecha transferencia',
                            'Mes de la transferencia':'Mes transferencia',
                            'RUT (si es persona jurídica)':'RUT',
                            'RUT, si receptor es persona jurídica':'RUT',
                            'Razón social de la persona jurídica':'Razón social',
                            'Razón social de la persona jurídica, si corresponde':'Razón social',
                            'Respaldo jurídico de la transferencia':'Respaldo transferencia',
                            'Utilidad o finalidad de la transferencia':'Finalidad transferencia',
                            'Región de la organización que recibe el aporte':'Región de la organización',
                            'Monto':'Monto transferencia'},
                   inplace=True)
    data[i]['Fecha transferencia'] = pd.to_datetime(data[i]['Fecha transferencia'])
    data[i]['Año transferencia'] = data[i]['Fecha transferencia'].dt.year #data[i].apply( lambda x: pd.to_datetime(x['Fecha transferencia']).year, axis=1)
    data[i]['Monto transferencia'] = data[i].apply( lambda x: float(x['Monto transferencia'].replace('.','')) if isinstance(x['Monto transferencia'], str) else float(x['Monto transferencia']), axis=1)

In [83]:
conicyt=pd.concat(data).reindex(columns=['Fecha transferencia','Año transferencia','Mes transferencia','Finalidad transferencia','Respaldo transferencia','Imputación presupuestaria','Monto transferencia','Unidad monetaria','RUT','Razón social','Apellido paterno','Apellido materno','Nombres','Región de la organización'])
conicyt=conicyt.sort_values('Fecha transferencia').reset_index(drop=True)
conicyt.head()

,Fecha transferencia,Año transferencia,Mes transferencia,Finalidad transferencia,Respaldo transferencia,Imputación presupuestaria,Monto transferencia,Unidad monetaria,RUT,Razón social,Apellido paterno,Apellido materno,Nombres,Región de la organización
0,2010-01-14,2010,Enero,Doctorado 2 2009-2 72100349,"DS. N° 664 del 29 de Diciembre de 2008, del Mi...",NaN,3705700.0,Pesos,NaN,NaN,BLANCO,JARA,JOSE,NaN
1,2010-01-14,2010,Enero,Doctorado 2 2009-2 72101165,"DS. N° 664 del 29 de Diciembre de 2008, del Mi...",NaN,3589397.0,Pesos,NaN,NaN,CASTAÑEDA,PEÑA,ROSELYN,NaN
2,2010-01-15,2010,Enero,DOCT 1 2009 72091017,"DS. N° 664 del 29 de Diciembre de 2008, del Mi...",NaN,11709209.0,Pesos,NaN,NaN,GARCIA HUIDOBRO,BECERRA,CRISTOBAL,NaN
3,2010-01-15,2010,Enero,BECA EXTRA. ASIG. MANUT. DOC EXT 13075559,Decreto supremo 29 del 29/12/1988,NaN,935221.0,Pesos,NaN,NaN,LLADSER,CALDERA,ALVARO FERNANDA,NaN
4,2010-01-15,2010,Enero,BECA EXTRA. ASIG. MANUT. DOC EXT 15651030,Decreto supremo 29 del 29/12/1988,NaN,625124.0,Pesos,NaN,NaN,NUÑEZ,CATALAN,MARCIA ANDREA,NaN


### Revisamos la columna Finalidad transferencia e identificamos que nos interesan aquellas entradas que contienen la frase "DOCTORADO 2010"

In [123]:
gv=conicyt['Finalidad transferencia'].str.contains('DOCTORADO 2010')
conicyt_doctorado=conicyt[gv].reset_index(drop=True)
conicyt_doctorado.head()

,Fecha transferencia,Año transferencia,Mes transferencia,Finalidad transferencia,Respaldo transferencia,Imputación presupuestaria,Monto transferencia,Unidad monetaria,RUT,Razón social,Apellido paterno,Apellido materno,Nombres,Región de la organización
0,2010-09-21,2010,Septiembre,BECAS NAC. ASIG. MENSU DOCTORADO 2010 14631527,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,560000.0,Pesos,NaN,NaN,AUFFARTH,II,ROBERT FREDERICK,XIII
1,2010-09-21,2010,Septiembre,BECAS NAC. ASIG. MENSU DOCTORADO 2010 15177658,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,3360000.0,Pesos,NaN,NaN,FUENTES,INZUNZA,RODRIGO ALEJANDRO,VIII
2,2010-09-21,2010,Septiembre,BECAS NAC. ASIG. MENSU DOCTORADO 2010 16094162,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,585000.0,Pesos,NaN,NaN,LOPEZ,ESPINOZA,PAULA CAROLINA,XIII
3,2010-09-21,2010,Septiembre,BECAS NAC. ASIG. MENSU DOCTORADO 2010 8376250,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,697000.0,Pesos,NaN,NaN,VILLAGRÁN,VALENZUELA,MAURICIO FELIPE,XIII
4,2010-09-23,2010,Septiembre,BECAS NAC. ASIG. MENSU DOCTORADO 2010 7220035,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,560000.0,Pesos,NaN,NaN,PEREZ,PRADO,MARIA FRANCISCA,XIII


### Una revisión de la columna Finalidad transferencia nos permite identificar que la palabra NACIONAL aparece abreviado como "NAC" y "NAC.". También notamos que el número de Folio es el último valor que aparece en el campo

In [95]:
conicyt_doctorado['Finalidad transferencia']=conicyt_doctorado.apply(lambda x: x['Finalidad transferencia'].replace('NAC.','NACIONAL').replace('NAC ','NACIONAL '), axis=1)
conicyt_doctorado['Folio']=conicyt_doctorado.apply(lambda x: int(x['Finalidad transferencia'].split(' ')[-1]) , axis=1)
conicyt_doctorado.head()

,Fecha transferencia,Año transferencia,Mes transferencia,Finalidad transferencia,Respaldo transferencia,Imputación presupuestaria,Monto transferencia,Unidad monetaria,RUT,Razón social,Apellido paterno,Apellido materno,Nombres,Región de la organización,Folio
0,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 1463...,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,560000.0,Pesos,NaN,NaN,AUFFARTH,II,ROBERT FREDERICK,XIII,14631527
1,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 1517...,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,3360000.0,Pesos,NaN,NaN,FUENTES,INZUNZA,RODRIGO ALEJANDRO,VIII,15177658
2,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 1609...,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,585000.0,Pesos,NaN,NaN,LOPEZ,ESPINOZA,PAULA CAROLINA,XIII,16094162
3,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 8376250,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,697000.0,Pesos,NaN,NaN,VILLAGRÁN,VALENZUELA,MAURICIO FELIPE,XIII,8376250
4,2010-09-23,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 7220035,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,560000.0,Pesos,NaN,NaN,PEREZ,PRADO,MARIA FRANCISCA,XIII,7220035


In [119]:
conicyt_query=conicyt_doctorado.query('Folio > 21100000').sort_values('Folio')
conicyt_doctorado_nomina=conicyt_query.groupby('Folio').first().reset_index()[['Folio','Apellido paterno','Apellido materno','Nombres']]

print('Nº de Doctorados 2010: ', len(conicyt_doctorado_nomina),'\n')
conicyt_doctorado_nomina.head()

Nº de Doctorados 2010:  498 



,Folio,Apellido paterno,Apellido materno,Nombres
0,21100005,JURFEST,RIVERO,SONIA PATRICIA
1,21100007,BASCUÑÁN,RODRIGUEZ,MARÍA LUZ
2,21100010,MUÑOZ,HERNANDEZ,LORETO ANDREA
3,21100020,PEREZ DE ARCE,JERIA,MIGUEL FELIPE
4,21100023,ZAMORA,SAA,JILBERTO ANTONIO


In [122]:
conicyt_doctorado_nomina.to_csv('data/conicyt doctorado 2010.csv', index=True, index_label='index')